In [ ]:
import torch
path = '/content/drive/My Drive/workshop_texts/' # write your own path for fine-tuned models

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer_path = 'ai-forever/rugpt3small_based_on_gpt2'
model_path = path + 'big_book_fine_gpt_model'

tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

# model = GPT2LMHeadModel.from_pretrained(model_path).to(device) # to use fine-tuned model
model = GPT2LMHeadModel.from_pretrained(tokenizer_path).to(device) # to use baseline model

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "sarahai/ruT5-base-summarizer"
tokenizer_summarize = T5Tokenizer.from_pretrained(model_name)
model_summarize = T5ForConditionalGeneration.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def generate(prompt, do_sample=True, num_beams=2, temperature=1.0, top_p=0.9, max_length=75): # TODO: play with hyperparameters

    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    model.eval()
    with torch.no_grad():
        out = model.generate(input_ids,
                            do_sample=do_sample,
                            num_beams=num_beams,
                            temperature=temperature,
                            top_p=top_p,
                            max_length=max_length,
                            repetition_penalty=1.6,
                            )

    return list(map(tokenizer.decode, out))[0]

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer_disc_path = 'cointegrated/rubert-tiny2'
tokenizer_disc = BertTokenizer.from_pretrained(tokenizer_disc_path)

model_disc_path = 'cointegrated/rubert-tiny2'
model_disc = BertForSequenceClassification.from_pretrained(model_disc_path)
model_disc = torch.load(path + 'BertClassifier2.pt', map_location=torch.device('cpu'))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
def get_score(text):
    input_ids = tokenizer_disc(text, return_tensors="pt")

    model_disc.eval()
    with torch.no_grad():
        outputs = model_disc(**input_ids)
        logits = outputs.logits
        logits = logits.detach().cpu().numpy()

    return int(np.argmax(logits, axis=1).flatten()[0])

In [ ]:
import re

def generate_book(prompt, length=1000, chunk_length=200):
    num_iter = length // chunk_length
    last_line = ""
    summary = prompt

    for i in range(num_iter):
      len_summary = len(re.findall(r'\w+', last_line))
      input_text = generate(summary, max_length=len_summary + chunk_length)

      while get_score(input_text) < 2:
          input_text = generate(summary, max_length=len_summary + chunk_length)

      if input_text.find(last_line) != -1:
          input_text = input_text[input_text.find(last_line) + len(last_line):]
      print(input_text, end='')


      input_text = input_text.splitlines()
      input_text = [x for x in input_text if x]

      #last_line = ' '.join(input_text[int(len(input_text)*0.8):])
      #summary = last_line

      last_line = input_text[-1]

      input_text = " ".join(input_text)

      input_ids = tokenizer_summarize(input_text, return_tensors="pt").input_ids.to(device)
      outputs = model_summarize.generate(input_ids, max_length=50, min_length=25, length_penalty=2.0, num_beams=4, early_stopping=True)
      summary = tokenizer_summarize.decode(outputs[0], skip_special_tokens=True)

      summary += last_line

    return

In [ ]:
generate_book("Глаза Степана Аркадьича весело заблестели, и он задумался, улыбаясь.")

Глаза Степана Аркадьича весело заблестели, и он задумался, улыбаясь.

– В чем же дело? – спросил он.

– Да вот в чем… – Степан Аркадьич достал из кармана свой кисет и протянул его Степану Аркадьичу.

– Ну, что ты хочешь сказать? – спросил Степан Аркадьич.

Степан Аркадьич кивнул головой.

– Ты знаешь, что за нами следят? – спросил он негромко.

Степан Аркадьич не ответил.

– Это я знаю, – сказал он.

– Что?

– То, что у нас есть.

Степан Аркадьич смотрел на него с сомнением.

– А ты откуда знаешь?

– Я видел тебя вчера.

– И что?

– Видел.

Степан Аркадьич молчал.

– У меня есть друзья, – сказал он

KeyboardInterrupt: 

In [ ]:
print(generate("Глаза Степана Аркадьича весело заблестели, и он задумался, улыбаясь.", max_length=300))

Глаза Степана Аркадьича весело заблестели, и он задумался, улыбаясь.

— Да, да, — подтвердил Степан Аркадьевич. — Я вижу, что вы хорошо поработали.

— Так точно, — подтвердил я.

— Ну, а теперь скажите мне, пожалуйста, зачем вы это сделали?

— Не знаю, — ответил я.

— Вот видите! — воскликнул Степан Аркадьевич. — А теперь скажите мне, зачем вы это сделали?

— Не знаю, — ответил я.

— А теперь скажите мне, зачем вы это сделали?

— Не знаю, — сказал я.

— А теперь скажите мне, зачем вы это сделали?

— А теперь скажите мне, зачем вы это сделали?

— Не знаю, — сказал я.

— А теперь скажите мне, зачем вы это сделали?

— Не знаю, — сказал я.

— А теперь скажите мне, зачем вы это сделали?

— Не знаю, — сказал я.

— А теперь скажите мне, зачем вы это сделали?

— Не знаю, — сказал я.

— А теперь скажите мне, зачем вы это сделали?

— Не знаю, — сказал я.

— А
